# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust, which has more than US$250 billion of assets under management.

In this project, I will create a Python script that accepts the value of my portfolio and tells me how many shares of each S&P 500 constituent I should purchase to get an equal-weight version of the index fund.

## Library Imports

The first step is to import the open-source software libraries that I will be using in this project. Below are the necessary imports:

```python
import numpy as np
import pandas as pd
import yfinance as yf


In [2]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np 
import pandas as pd 
import math 
import requests
import yfinance as yf

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Now it's time to import these stocks to our Jupyter Notebook file.

In [5]:
df = pd.read_csv('constituents.csv')
Tickers = df['Symbol']
Tickers

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

## Testing Yfinance call for specific fields 

The yf call that we executed in the last code block seems to have all teh required data to build an Equal weighted S&P 500 fund. Testing MarketCap and price endpoints 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
tickerData = yf.Ticker('AAPL')

In [7]:
marketCap = tickerData.info['marketCap']
marketCap 
previousClose = tickerData.info['previousClose']
previousClose

224.31

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
# Assuming df is your initial DataFrame containing a column 'Ticker' with the ticker symbols
#df = pd.DataFrame({'Ticker': ['AAPL', 'GOOGL', 'MSFT']})  # Example tickers

# Initialize an empty list to collect data
market_cap_data = []

# Fetch market capitalization for each ticker and append to the list
for ticker in Tickers:
    try:
        market_cap = yf.Ticker(ticker).info['marketCap']
        market_cap_data.append({'ticker': ticker, 'marketC': market_cap})
    except:
        continue

# Convert the list to a DataFrame
marketCap_df = pd.DataFrame(market_cap_data)
print(marketCap_df)

    ticker       marketC
0      MMM   58086305792
1      AOS   13025762304
2      ABT  180434419712
3     ABBV  306572689408
4      ACN  207652569088
..     ...           ...
496    XYL   33782564864
497    YUM   36220690432
498   ZBRA   16792034304
499    ZBH   22697969664
500    ZTS   82479882240

[501 rows x 2 columns]


In [9]:
print(marketCap_df.head)

<bound method NDFrame.head of     ticker       marketC
0      MMM   58086305792
1      AOS   13025762304
2      ABT  180434419712
3     ABBV  306572689408
4      ACN  207652569088
..     ...           ...
496    XYL   33782564864
497    YUM   36220690432
498   ZBRA   16792034304
499    ZBH   22697969664
500    ZTS   82479882240

[501 rows x 2 columns]>


In [10]:
Columns_final = ['Ticker','Stock Price','Market Capitalization','Num Shares to Buy']
Output_df = pd.DataFrame(columns=Columns_final)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [11]:
priceData =[]
for ticker in Tickers:
    try:
        market_cap = yf.Ticker(ticker).info['marketCap']
        market_cap_data.append(market_cap)
        price = yf.Ticker(ticker).info['previousClose']
        priceData.append(price)
    except:
        continue

In [12]:
Output_df['Stock Price'] = priceData
Output_df['Market Capitalization'] = market_cap_data[0:501]
Output_df['Ticker'] = Tickers

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [32]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [33]:
# Splitting tickers into chunks of 100
symbol_groups = list(chunks(Output_df['Ticker'], 100))
symbol_strings = [','.join(group) for group in symbol_groups]

# Initializing the final dataframe
final_dataframe = pd.DataFrame(columns=Columns_final)

# Iterating through each group of symbols and fetching data in batches
for symbol_string in symbol_strings:
    # Fetch data for all tickers in the current batch
    data = yf.download(tickers=symbol_string, period="1d", group_by='ticker', auto_adjust=False)

    for symbol in symbol_string.split(','):
        try:
            if isinstance(data, pd.DataFrame) and symbol in data.columns.levels[0]:
                previous_close = data[symbol]['Close'].iloc[-1]
            elif isinstance(data, pd.DataFrame):
                previous_close = data['Close'].iloc[-1]
            else:
                previous_close = data['Close'].iloc[-1] if 'Close' in data else 'N/A'
                
            ticker_data = yf.Ticker(symbol)
            market_cap = ticker_data.info.get('marketCap', 'N/A')
            
            new_row = pd.DataFrame([[symbol, previous_close, market_cap, 'N/A']], columns=Columns_final)
            final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
print(final_dataframe)

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
496    WYNN
497     XEL
498     XYL
499     YUM
500    ZBRA
Name: Ticker, Length: 501, dtype: object
[0      MMM
1      AOS
2      ABT
3     ABBV
4      ACN
      ... 
95      CE
96     COR
97     CNC
98     CNP
99      CF
Name: Ticker, Length: 100, dtype: object, 100    CHRW
101     CRL
102    SCHW
103    CHTR
104     CVX
       ... 
195     FIS
196    FITB
197    FSLR
198      FE
199      FI
Name: Ticker, Length: 100, dtype: object, 200     FMC
201       F
202    FTNT
203     FTV
204    FOXA
       ... 
295       L
296     LOW
297    LULU
298     LYB
299     MTB
Name: Ticker, Length: 100, dtype: object, 300     MRO
301     MPC
302    MKTX
303     MAR
304     MMC
       ... 
395     RTX
396       O
397     REG
398    REGN
399      RF
Name: Ticker, Length: 100, dtype: object, 400     RSG
401     RMD
402    RVTY
403     ROK
404     ROL
       ... 
495     WTW
496    WYNN
497     XEL
498     XYL
499     YUM
Name: Tic

[*********************100%%**********************]  100 of 100 completed

2 Failed downloads:
['BF.B']: YFInvalidPeriodError("%ticker%: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']")
['BRK.B']: YFChartError('%ticker%: No data found, symbol may be delisted')
/var/folders/g4/d2_9vd6n0cx4k51m5kmh8f2m0000gn/T/ipykernel_10330/783701940.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
[*********************100%%**********************]  100 of 100 completed
[*********************100%%**********************]  100 of 100 completed
[*********************100%%**********************]  100 of 100 completed
[**************

Error fetching data for ZBRA: 'Index' object has no attribute 'levels'
    Ticker  Stock Price Market Capitalization Num Shares to Buy
0      MMM   104.970001           58086305792               N/A
1      AOS    88.809998           13025762304               N/A
2      ABT   103.720001          180434419712               N/A
3     ABBV   173.610001          306572689408               N/A
4      ACN   331.510010          207652569088               N/A
..     ...          ...                   ...               ...
495    WTW   265.440002           27137523712               N/A
496   WYNN    83.540001            9362411520               N/A
497    XEL    55.740002           30971318272               N/A
498    XYL   139.339996           33782564864               N/A
499    YUM   128.610001           36220690432               N/A

[500 rows x 4 columns]


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [17]:
user_input = input("Please enter something: ")
val = float(user_input)
print(f"You entered: {user_input}")

You entered: 100000


In [36]:
position_size = val/500
num_shares = []
for price in final_dataframe['Stock Price']:
    num_shares.append(position_size/price)
final_dataframe['Num Shares to Buy']= pd.Series(num_shares)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Num Shares to Buy
0,MMM,104.970001,58086305792,1.905306
1,AOS,88.809998,13025762304,2.251999
2,ABT,103.720001,180434419712,1.928268
3,ABBV,173.610001,306572689408,1.152007
4,ACN,331.510010,207652569088,0.603300
...,...,...,...,...
495,WTW,265.440002,27137523712,0.753466
496,WYNN,83.540001,9362411520,2.394063
497,XEL,55.740002,30971318272,3.588087
498,XYL,139.339996,33782564864,1.435338


## Saving Our Excel Output using the openyxl engine

In [52]:
#writer.close()
final_dataframe.to_excel('Trade_Details.xlsx', engine='openpyxl')